# MNIST Dataset Classification Task
## Charis Filis Academic-Id: 9449

### Import dependecies 

In [1]:
from __future__ import absolute_import, division, print_function
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
from tensorflow.keras.datasets import mnist
from tensorflow.keras import regularizers
from tensorflow.keras import initializers
from keras.utils import np_utils
import keras_tuner as kt
import sympy
import numpy as np
import seaborn as sns
import collections
# visualization tools
%matplotlib inline
import matplotlib.pyplot as plt

In [2]:
# Building a toy network Simple Perceptron
x = Input(tf.float32,[None,784])
W = tf.Variable(tf.zeros[10])
a = tf.Variable(tf.zeros[10])
pred = tf.matmul((x,W)) + b 
pred= model(x,pred)


correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))
loss_f = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y,logits=y))

NameError: name 'Input' is not defined

In [ ]:
from random import shuffle
def get_images(batch_size, n_iterations):
    return [mnist.train.next_batch(batch_size=batch_size,shuffle=True) for dummy_index in range(n_iterations) ]

In [ ]:
from random import shuffle
def get_images(batch_size, n_iterations):
    return [mnist.train.next_batch(batch_size=batch_size,shuffle=True) for dummy_index in range(n_iterations) ]

In [ ]:
def minibatch_training(optimization_t, learning_r, train_set, minibatch_size):
    accuracy_buf = np.zeros(len(train_set))
    cross_entr_loss_buf = np.zeros(len(train_set))
    
    opt = optimization_t(learning_r)
    # Retrieve Trainable variables
    tvs = tf.trainable_variables()
    # Create Buffer for accumulating gradients/weights 
    accum_weights = [tf.Variable(tv.initialized_value(),trainable=False)] for tv in tvs
    # Operation  to initialize accum_weights to zero
    zero_ops  = [tv.assign(tf.zeros_like(tv)) for tv in accum_vars]
    
    # Operation to compute gradients from one minibatch
    gvs = opt.com 

### Data Preprocessing

In [10]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

# Rescale the images from [0,255] to the [0.0,1.0] range.
x_train, x_test = x_train[..., np.newaxis]/255.0, x_test[..., np.newaxis]/255.0
n_classes = 10;
n_features = 784; # features are the pixels of each image
print("Number of original training examples:", len(x_train))
print("Number of original test examples:", len(x_test))
print("Training data shape:",x_train.shape)
print("Training y train shape", y_train.shape)
print("Testing data shape:",x_test.shape)
# Training parameters.
learning_rate = 0.1
training_steps = 2000
batch_size = 256
display_step = 100
# Network parameters.
n_hidden_1 = 128 # 1st layer number of neurons.
n_hidden_2 = 256 # 2nd layer number of neurons.

Number of original training examples: 60000
Number of original test examples: 10000
Training data shape: (60000, 28, 28, 1)
Training y train shape (60000,)
Testing data shape: (10000, 28, 28, 1)


In [11]:
# Proper minibatch Split
# Instantiate an optimizer.
optimizer = keras.optimizers.SGD(learning_rate=1e-3)
# Instantiate a loss function.
loss_fn = keras.losses.SparseCategoricalCrossentropy(from_logits=True)

# Prepare the training dataset.
batch_size = 64
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
x_train = np.reshape(x_train, (-1, 784))
x_test = np.reshape(x_test, (-1, 784))

# Reserve 10,000 samples for validation.
x_val = x_train[-10000:]
y_val = y_train[-10000:]
x_train = x_train[:-10000]
y_train = y_train[:-10000]

# Prepare the training dataset.
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_dataset = train_dataset.shuffle(buffer_size=1024).batch(batch_size)

# Prepare the validation dataset.
val_dataset = tf.data.Dataset.from_tensor_slices((x_val, y_val))
val_dataset = val_dataset.batch(batch_size)


# Part 1    Different MLP Architectures Comparison

# model_default : Default settings for Dense Sequential Model

In [14]:
model_default = keras.Sequential()
# Input Layer
# 1st hidden Layer
model_default.add(layers.Dense(n_hidden_1,input_shape=(n_features,), activation='relu'))
# 2nd hidden Layer
model_default.add(layers.Dense(n_hidden_2,activation='relu'))
# Output Layer
model_default.add(layers.Dense(10))
model_default.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_8 (Dense)             (None, 128)               100480    
                                                                 
 dense_9 (Dense)             (None, 256)               33024     
                                                                 
 dense_10 (Dense)            (None, 10)                2570      
                                                                 
Total params: 136,074
Trainable params: 136,074
Non-trainable params: 0
_________________________________________________________________


In [ ]:
def default_Model(Model):
    
    # Set Layers
    model_default = keras.Sequential()
    # Input Layer
    # 1st hidden Layer
    model_default.add(layers.Dense(n_hidden_1,input_shape=(n_features,), activation='relu'))
    # 2nd hidden Layer
    model_default.add(layers.Dense(n_hidden_2,activation='relu'))
    # Output Layer
    model_default.add(Dense(10, activation='softmax', kernel_initializer=initializer))
    model_defaultl.summary()
    model_default.compile(loss=keras.losses.CategoricalCrossentropy(), optimizer=keras.optimizers.Adam(learning_rate=0.001), metrics=['accuracy'])
    return model

#### * Define Cross Entropy Loss
#### * Define Accuracy Metric

In [ ]:
# Cross-entropy loss function
# With this implementation we dont need extra softmax layer
def cross_entropy_loss(x,y):
    # Convert labels to int64 for this function
    y = tf.case(y,tf.int64)
    # Apply Softmax to logits and compute cross-entropy 
    loss = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y,logits=x)
    # Average loss across the batch
    return tf.reduce_mean(loss)

# Accuracy metric
def accuracy(y_pred,y_gt):
    # Predicted class is the index of highest score in prediction vector
    correct_prediction = tf.equal(tf.argmax(y_pred,1), tf.cast(y_gt,tf.int64))
    # take mean accros correct_prediction vector
    return tf.reduce_mean(tf.cast(correct_prediction,tf.float32),axis=1)

# Optimizer 
optimizer = tf.optimizers.RMSprop(learning_rate=0.001,rhp=0.9,momentum='0', epsilon=1e-7,centered=False,name="RMSpropOptim")
optimizer = tf.optimizers.SGD(learning_rate=///)